In [1]:
from itertools import product
import dolphindb as ddb

In [19]:
%matplotlib inline

In [102]:
ddb_conn = ddb.session()
ddb_conn.connect("localhost", 8848, userid="admin", password="123456")

True

In [103]:
load_table = """
db_QUOTE = database("dfs://daily_quote");  //委託簿
getTables(db_QUOTE);
tickQuote=loadTable(db_QUOTE,`tickQuote);

db_TSE=database("dfs://daily_tse");  //成交行情
getTables(db_TSE);
tickTSE=loadTable(db_TSE,`tickTSE);

db_TFE=database("dfs://daily_tfe");  //成交行情
getTables(db_TFE);
tickTFE=loadTable(db_TFE,`tickTFE);

db_IDX = database("dfs://daily_index");  //指數
getTables(db_IDX);
tickIDX=loadTable(db_IDX,`tickIDX);
"""

In [104]:
ddb_conn.run(load_table)

In [105]:
df_free = ddb_conn.run("select sum(freeSize) as freeSize , sum(blockSize) as allocatedSize from mem();")
print(df_free["freeSize"]/ df_free["allocatedSize"])
df_free

0    0.997941
dtype: float64


,freeSize,allocatedSize
0,104641728,104857600


In [81]:
ddb_conn.run("t = select count(*) from tickTFE group by Date;")

In [88]:
df_free = ddb_conn.run("select sum(freeSize) as freeSize , sum(blockSize) as allocatedSize from mem();")
print(df_free["freeSize"]/ df_free["allocatedSize"])
df_free

0    0.544083
dtype: float64


,freeSize,allocatedSize
0,349157856,641736704


In [87]:
ddb_conn.run("t = NULL")

In [84]:
ddb_conn.run('undef all;')

In [89]:
df_free = ddb_conn.run("select sum(freeSize) as freeSize , sum(blockSize) as allocatedSize from mem();")
print(df_free["freeSize"]/ df_free["allocatedSize"])
df_free

0    0.544083
dtype: float64


,freeSize,allocatedSize
0,349157856,641736704


In [97]:
df_free = ddb_conn.run("select sum(freeSize) as freeSize , sum(blockSize) as allocatedSize from mem();")
print(df_free["freeSize"]/ df_free["allocatedSize"])
df_free

0    0.099794
dtype: float64


,freeSize,allocatedSize
0,439081392,4399890432


In [106]:
ddb_conn.run("select count(*), first(AskPrice1), first(BidPrice1) from tickQuote group by Date;")

,Date,count,first_AskPrice1
0,2019-03-04,26080692,0.0000
1,2019-03-05,27863933,0.4000
2,2019-03-06,26521977,186.0000
3,2019-03-07,28836608,0.0000
4,2019-03-08,29659594,0.0000
...,...,...,...
203,2019-11-05,43043862,0.0000
204,2019-11-06,44155493,101.0000
205,2019-11-07,42242580,0.0128
206,2019-11-08,42871202,11.5000


In [108]:
ddb_conn.run('select count(*), first(AskPrice1), first(BidPrice1) '
             'from tickQuote where Code="TXFK9" group by Date;')

,Date,count,first_AskPrice1,first_BidPrice1
0,2019-08-22,157661,10888.0,10014.0
1,2019-08-23,176602,10471.0,10395.0
2,2019-08-24,41975,10391.0,10319.0
3,2019-08-26,180933,0.0,10125.0
4,2019-08-27,87531,10330.0,10301.0
...,...,...,...,...
60,2019-11-05,215741,11600.0,11599.0
61,2019-11-06,205998,11634.0,11633.0
62,2019-11-07,225844,11636.0,11635.0
63,2019-11-08,216979,11669.0,11668.0


In [113]:
ddb_conn.run('select count(*), first(AskPrice1), first(BidPrice1) '
             'from tickQuote where Code="TXFK9" group by concatDateTime(Date, Time) as dt;')

,dt,count,first_AskPrice1,first_BidPrice1
0,2019-08-22 08:31:25.000,1.0,10888.0,10014.0
1,2019-08-22 08:31:30.171,1.0,10888.0,10014.0
2,2019-08-22 08:31:35.171,1.0,10888.0,10014.0
3,2019-08-22 08:31:40.171,1.0,10888.0,10014.0
4,2019-08-22 08:31:45.171,1.0,10888.0,10014.0
...,...,...,...,...
8920072,2019-11-09 04:59:55.340,1.0,11587.0,11585.0
8920073,2019-11-09 04:59:55.965,1.0,11587.0,11585.0
8920074,2019-11-09 04:59:56.715,1.0,11587.0,11586.0
8920075,2019-11-09 04:59:57.215,1.0,11587.0,11586.0


In [77]:
df_free = ddb_conn.run("select sum(freeSize) as freeSize , sum(blockSize) as allocatedSize from mem();")
print(df_free["freeSize"]/ df_free["allocatedSize"])
df_free

0    0.998254
dtype: float64


,freeSize,allocatedSize
0,104674560,104857600


In [75]:
df_free = ddb_conn.run("select sum(freeSize) as freeSize , sum(blockSize) as allocatedSize from mem();")
print(df_free["freeSize"]/ df_free["allocatedSize"])
df_free

0    0.098114
dtype: float64


,freeSize,allocatedSize
0,431692064,4399890432


In [74]:
ddb_conn.run("undef(all);")

In [57]:
quote_cols = [('').join(c) for c in product(["Ask", "Bid"], 
                                            ["Price", "Volume"], 
                                            map(str, range(1, 6)))]
quote_cols_str = (", ").join(quote_cols)

In [58]:
ddb_conn.run('select concatDateTime(Date,Time) as DateTime, Code, {0} '
             'from tickQuote where Date=2019.11.09 and Code="TXFK9";'.format(quote_cols_str))

,DateTime,Code,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,AskVolume1,AskVolume2,AskVolume3,...,BidPrice1,BidPrice2,BidPrice3,BidPrice4,BidPrice5,BidVolume1,BidVolume2,BidVolume3,BidVolume4,BidVolume5
0,2019-11-09 00:00:00.090,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,26.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,24.0,34.0,32.0,46.0,40.0
1,2019-11-09 00:00:00.840,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,24.0,34.0,37.0,46.0,40.0
2,2019-11-09 00:00:00.965,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,25.0,34.0,37.0,46.0,40.0
3,2019-11-09 00:00:01.465,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,25.0,34.0,38.0,46.0,40.0
4,2019-11-09 00:00:01.715,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,37.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,25.0,36.0,38.0,46.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21773,2019-11-09 04:59:55.340,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,9.0,2.0,7.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,10.0,5.0,5.0,7.0,9.0
21774,2019-11-09 04:59:55.965,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,8.0,2.0,7.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,10.0,5.0,5.0,7.0,9.0
21775,2019-11-09 04:59:56.715,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,8.0,2.0,7.0,...,11586.0,11585.0,11584.0,11583.0,11582.0,2.0,10.0,5.0,5.0,7.0
21776,2019-11-09 04:59:57.215,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,8.0,2.0,7.0,...,11586.0,11585.0,11584.0,11583.0,11582.0,2.0,10.0,5.0,5.0,7.0


In [63]:
ddb_conn.run('select count(*) from tickQuote where Date=2019.11.08 group by Code;').sort_values(
    'count', ascending=False)

,Code,count
669,MXFK9,227380
31,TXFK9,216979
45,UNFC0,175254
320,UNFL9,175089
567,MX2K9,168135
...,...,...
9745,HCFL9/C0,1
20859,KWFL9/C0,1
17221,FWFL9/C0,1
8614,DIFL9/I0,1


In [100]:
_63.head(125)

,Code,count
669,MXFK9,227380
31,TXFK9,216979
45,UNFC0,175254
320,UNFL9,175089
567,MX2K9,168135
...,...,...
206,TXO11400A0,27205
579,TX211350W9,26546
400,TXO11700M0,26300
479,TXO11600L9,25916


In [101]:
ddb_conn.run('select concatDateTime(Date,Time) as DateTime, Code, {0} '
             'from tickQuote where Date=2019.11.08 and Code="TXO11600L9";'.format(quote_cols_str))

RuntimeError: <Server Exception> in run: Out of memory

In [31]:
ddb_conn.run("schema(tickQuote);")

{'chunkPath': None, 'colDefs':           name typeString  typeInt comment
 0    AskPrice1     DOUBLE       16        
 1    AskPrice2     DOUBLE       16        
 2    AskPrice3     DOUBLE       16        
 3    AskPrice4     DOUBLE       16        
 4    AskPrice5     DOUBLE       16        
 5   AskVolume1     DOUBLE       16        
 6   AskVolume2     DOUBLE       16        
 7   AskVolume3     DOUBLE       16        
 8   AskVolume4     DOUBLE       16        
 9   AskVolume5     DOUBLE       16        
 10   BidPrice1     DOUBLE       16        
 11   BidPrice2     DOUBLE       16        
 12   BidPrice3     DOUBLE       16        
 13   BidPrice4     DOUBLE       16        
 14   BidPrice5     DOUBLE       16        
 15  BidVolume1     DOUBLE       16        
 16  BidVolume2     DOUBLE       16        
 17  BidVolume3     DOUBLE       16        
 18  BidVolume4     DOUBLE       16        
 19  BidVolume5     DOUBLE       16        
 20        Date       DATE        6        
 2

In [12]:
ddb_conn.run("select * from tickTFE where Date=2019.11.09;")

,Code,Date,Time,Close,Volume,TickType,Index
0,UDFC0,2019-11-09,1970-01-01 00:34:18.999,27555.0,1.0,1.0,0
1,UDFC0,2019-11-09,1970-01-01 00:48:17.269,27567.0,1.0,1.0,0
2,UDFC0,2019-11-09,1970-01-01 00:48:17.419,27567.0,1.0,0.0,0
3,UDFC0,2019-11-09,1970-01-01 00:48:19.568,27566.0,1.0,2.0,0
4,UDFC0,2019-11-09,1970-01-01 00:48:20.849,27566.0,1.0,0.0,0
...,...,...,...,...,...,...,...
11585,TX211750W9,2019-11-09,1970-01-01 01:11:14.466,157.0,1.0,2.0,0
11586,TX211750W9,2019-11-09,1970-01-01 01:45:30.694,161.0,4.0,1.0,0
11587,TX211750W9,2019-11-09,1970-01-01 01:50:13.423,164.0,1.0,1.0,0
11588,TX211750W9,2019-11-09,1970-01-01 01:58:46.444,161.0,1.0,2.0,0


In [17]:
ddb_conn.run('select Code, concatDateTime(Date,Time) as Datetime, '
             'Close, Volume, TickType, Index '
             'from tickTFE where Code="TXFK9";')

,Code,Datetime,Close,Volume,TickType,Index
0,TXFK9,2019-08-22 09:00:35.832,10496.0,1.0,1.0,0
1,TXFK9,2019-08-22 09:01:09.919,10500.0,1.0,1.0,0
2,TXFK9,2019-08-22 09:01:16.597,10500.0,1.0,1.0,0
3,TXFK9,2019-08-22 09:01:47.446,10496.0,1.0,2.0,0
4,TXFK9,2019-08-22 09:02:55.127,10498.0,1.0,1.0,0
...,...,...,...,...,...,...
743154,TXFK9,2019-11-09 04:59:52.538,11587.0,1.0,1.0,0
743155,TXFK9,2019-11-09 04:59:55.000,11586.0,1.0,2.0,0
743156,TXFK9,2019-11-09 04:59:55.001,11585.0,1.0,2.0,0
743157,TXFK9,2019-11-09 04:59:55.001,11585.0,1.0,2.0,0
